In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Concatenate,Dense, Subtract,Flatten,Lambda
import matplotlib.pyplot as plt
import cv2
from  scipy import ndimage

In [ ]:
!curl -O http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xzvf lfw.tgz

In [ ]:
def show(image, label):
    plt.figure()
    plt.imshow(image[0])
    # plt.title(labels[label])
    plt.title(label)
    plt.axis('off')

In [ ]:
import os
import random
BASE_PATH = 'lfw'

labels = [i for i in os.listdir(BASE_PATH)]
paths_with_more_than_2_images = []

train_labels = [i for i in labels if len(os.listdir(os.path.join(BASE_PATH,i)))>=2]
test_labels = list(set(labels)-set(train_labels))
print(len(train_labels),len(test_labels))

In [1]:
#Building model

In [ ]:
from tensorflow.keras.optimizers import RMSprop
opt = RMSprop()

In [ ]:
def triplet_loss(y_true,x, alpha = 0.2):
    # Triplet Loss function
    anchor = x[:,0:64]
    positive = x[:,64:128]
    negative = x[:,128:]
    
    # distance between the anchor and the positive
    pos_dist = tf.keras.backend.sum(tf.keras.backend.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = tf.keras.backend.sum(tf.keras.backend.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = tf.keras.backend.maximum(basic_loss,0.0)
    return loss

In [ ]:
def complete_model(base_model):
    input1 = Input(shape=(224,224,3),name='Input_1')
    input2 = Input(shape=(224,224,3),name="Input_2")
    input3 = Input(shape=(224,224,3),name='Input_3')
    A = base_model(input1)
    P = base_model(input2)
    N = base_model(input3)
    output = Concatenate(name='Merged')([A,P,N])
    model = tf.keras.Model(inputs=[input1, input2, input3], outputs=output)
    model.compile(loss=triplet_loss, optimizer=tf.keras.optimizers.RMSprop(0.001))

    return model

In [ ]:
base_model=buildEmbeddingModel()
model = complete_model(base_model)

In [1]:
#Training model

In [ ]:
model.fit(tripletDataGenerator(train_labels,training=True),epochs=20,callbacks=[TensorBoard('./logs'),ModelCheckpoint('./model',save_best_only=True)],steps_per_epoch=200,verbose=1,validation_data=tripletDataGenerator(labels,training=False),validation_steps=30)

In [ ]:
base_model.save('base_model.h5')
base_model.save('saved_model/base_model')